In [6]:
import requests
import time

transaction_fee = 0.005
major_currencies = ["DOGE","LTC"]
ignore_currencies = []
minimum_transaction = {"BTC":0.0005, "USDT":1, "NZDT":1, "LTC":0.01, "DOGE":100}

try:
    markets = requests.get("https://www.cryptopia.co.nz/api/GetMarkets").json()
    if not markets["Success"]:
        exit()
except:
    exit()


class graph:
    """
    Directed Cyclical Digraph
    """
    def __init__(self):
        self.currencies = {} #dictionary that maps currency name to currency objects

        for i in markets['Data']:
            if i["Volume"] > 0:
                coin1, coin2 = i["Label"].split("/")
                if coin1 in ignore_currencies or coin2 in ignore_currencies:
                    continue
                if coin1 not in self.currencies.keys():
                    self.currencies[coin1] = node(coin1)
                if coin2 not in self.currencies.keys():
                    self.currencies[coin2] = node(coin2)

                self.currencies[coin1].addChildren(self.currencies[coin2], i["BidPrice"])
                self.currencies[coin2].addChildren(self.currencies[coin1], 1/i["AskPrice"])

    def __repr__(self):
        return str(self.currencies)

class node:
    def __init__(self, name):
        self.name = name
        self.children = {} #dictionary that maps transactions to their price (e.g, if 1 unit of X gives 10 units of Y then we have {Y:10})

    def __repr__(self):
        return self.name

    def addChildren(self, newNode, conversion_rate):
        self.children[newNode] = conversion_rate

    def printChildren(self):
        print(str(self.children))

accumulator = {}
calls = 0

def find_good_cycles(graph):
    for currency in major_currencies:
        dfs([graph.currencies[currency]], 1, max_depth = 4)

def dfs(visited, conversion_product, max_depth):
    global calls

    if(max_depth <= 0):
        return
    for nextNode, conversion_rate in visited[-1].children.items():
        p = (1 - transaction_fee)*conversion_rate*conversion_product

        if nextNode in visited:
            if len(visited) > 3 and visited[0] == nextNode and p > 1:
                calls += 1
                accumulator[tuple(visited + [nextNode])] = p
        else:
            newVisited = list(visited)
            newVisited.append(nextNode)
            dfs(newVisited, p, max_depth - 1)

g = graph()
start = time.time()
find_good_cycles(g)
end = time.time()

sorted_list = sorted(accumulator.items(), key=lambda kv: kv[1], reverse=True)
for item in sorted_list:
  print(item)

for item in sorted_list[0:2]:
    valid = True
    value = 0
    volume = 0

    for i in range(len(item[0])-1):
        coin1 = item[0][i]
        coin2 = item[0][i+1]

        orders = requests.get("https://www.cryptopia.co.nz/api/GetMarketOrderGroups/%s_%s/3" % (coin1, coin2)).json()
        if orders["Data"] == None:
            orders = requests.get("https://www.cryptopia.co.nz/api/GetMarketOrderGroups/%s_%s/3" % (coin2, coin1)).json()


        c1, c2 = orders["Data"][0]["Buy"][0]["Label"].split("_")
        ## Fulfill Sell Order
        if str(coin1) == c2:
            value = (1/orders["Data"][0]["Sell"][0]["Price"])
            if i == 0:
                volume = orders["Data"][0]["Sell"][0]["Volume"]
            else:
                if orders["Data"][0]["Sell"][0]["Volume"] > volume*value:
                    volume = volume*value
                else:
                    volume = orders["Data"][0]["Sell"][0]["Volume"]
        else:
            value = (orders["Data"][0]["Buy"][0]["Price"])
            if i == 0:
                volume = orders["Data"][0]["Buy"][0]["Total"]
            else:
                if orders["Data"][0]["Buy"][0]["Total"] > volume*value:
                    volume = volume*value
                else:
                    volume = orders["Data"][0]["Buy"][0]["Total"]

        value = (1 - transaction_fee) * value

#         print("BUY ORDER" + str(orders["Data"][0]["Buy"][0]))
#         print("SELL ORDER" + str(orders["Data"][0]["Sell"][0]))
#         print(value, volume)
#     print(item, valid)

print("Number of cycles found: ", len(accumulator))
print("Runtime", end - start)

((DOGE, SOON, BTC, HAC, DOGE), 1.2325059810457037)
((DOGE, MUSIC, BTC, HAC, DOGE), 1.2004955608342447)
((DOGE, LINX, BTC, HAC, DOGE), 1.1867696777191787)
((DOGE, DZC, BTC, HAC, DOGE), 1.1780493203060416)
((DOGE, LTC, BTC, HAC, DOGE), 1.1775395600399332)
((LTC, BTC, HAC, DOGE, LTC), 1.1775395600399332)
((DOGE, ETN, BTC, HAC, DOGE), 1.177356773525967)
((DOGE, ETH, BTC, HAC, DOGE), 1.176130759520228)
((DOGE, XEM, BTC, HAC, DOGE), 1.1742777913012912)
((DOGE, ZEC, BTC, HAC, DOGE), 1.1737424697375582)
((DOGE, USDT, BTC, HAC, DOGE), 1.1736214982461892)
((DOGE, XVG, BTC, HAC, DOGE), 1.1734139103156274)
((DOGE, TRX, BTC, HAC, DOGE), 1.173100456279922)
((DOGE, RVN, BTC, HAC, DOGE), 1.1730779054535871)
((DOGE, GBX, BTC, HAC, DOGE), 1.172747465912849)
((DOGE, CNET, BTC, HAC, DOGE), 1.17221077540246)
((DOGE, BCH, BTC, HAC, DOGE), 1.171648218716391)
((DOGE, ETC, BTC, HAC, DOGE), 1.1709476459265382)
((DOGE, EOS, BTC, HAC, DOGE), 1.170608278272107)
((DOGE, TUSD, BTC, HAC, DOGE), 1.1695275196766275)
((

In [0]:
import requests
import time

transaction_fee = 0.005
major_currencies = ["DOGE"]
ignore_currencies = []
minimum_transaction = {"BTC":0.0005, "USDT":1, "NZDT":1, "LTC":0.01, "DOGE":100}

try:
    markets = requests.get("https://www.cryptopia.co.nz/api/GetMarkets").json()
    if not markets["Success"]:
        exit()
except:
    exit()


class graph:
    """
    Directed Cyclical Digraph
    """
    def __init__(self):
        self.currencies = {} #dictionary that maps currency name to currency objects

        for i in markets['Data']:
            if i["Volume"] > 0:
                coin1, coin2 = i["Label"].split("/")
                if coin1 in ignore_currencies or coin2 in ignore_currencies:
                    continue
                if coin1 not in self.currencies.keys():
                    self.currencies[coin1] = node(coin1)
                if coin2 not in self.currencies.keys():
                    self.currencies[coin2] = node(coin2)

                self.currencies[coin1].addChildren(self.currencies[coin2], i["BidPrice"])
                self.currencies[coin2].addChildren(self.currencies[coin1], 1/i["AskPrice"])

    def __repr__(self):
        return str(self.currencies)

class node:
    def __init__(self, name):
        self.name = name
        self.children = {} #dictionary that maps transactions to their price (e.g, if 1 unit of X gives 10 units of Y then we have {Y:10})

    def __repr__(self):
        return self.name

    def addChildren(self, newNode, conversion_rate):
        self.children[newNode] = conversion_rate

    def printChildren(self):
        print(str(self.children))

accumulator = {}
calls = 0

def find_good_cycles(graph):
    for currency in major_currencies:
        dfs([graph.currencies[currency]], 1, max_depth = 4)

def dfs(visited, conversion_product, max_depth):
    global calls

    if(max_depth <= 0):
        return
    for nextNode, conversion_rate in visited[-1].children.items():
        p = (1 - transaction_fee)*conversion_rate*conversion_product

        if nextNode in visited:
            if len(visited) > 3 and visited[0] == nextNode and p > 1:
                calls += 1
                accumulator[tuple(visited + [nextNode])] = p
        else:
            newVisited = list(visited)
            newVisited.append(nextNode)
            dfs(newVisited, p, max_depth - 1)

g = graph()
start = time.time()
find_good_cycles(g)
end = time.time()

sorted_list = sorted(accumulator.items(), key=lambda kv: kv[1], reverse=False)

for item in sorted_list[2:]:
    valid = True
    value = 0
    volume = 0

    for i in range(len(item[0])):
        if i == len(item[0])-1:
            print("great success")
            print(item)
            quit()
            break
        else:
            coin1 = item[0][i]
            coin2 = item[0][i+1]

            orders = requests.get("https://www.cryptopia.co.nz/api/GetMarketOrderGroups/%s_%s/3" % (coin1, coin2)).json()
            if orders["Data"] == None:
                orders = requests.get("https://www.cryptopia.co.nz/api/GetMarketOrderGroups/%s_%s/3" % (coin2, coin1)).json()

            c1, c2 = orders["Data"][0]["Buy"][0]["Label"].split("_")
        ## Fulfill Sell Order
        if i == 0:
            if str(coin1) == c2:
                volume = orders["Data"][0]["Sell"][0]["Volume"]
            else:
                volume = orders["Data"][0]["Buy"][0]["Total"]
        elif str(coin1) == c2:
            value = (1/orders["Data"][0]["Sell"][0]["Price"])

            print("sell" + str(volume))
            if volume < minimum_transaction[c2]:
                break
            if orders["Data"][0]["Sell"][0]["Volume"] > volume*value:
                volume = volume*value
            else:
                volume = orders["Data"][0]["Sell"][0]["Volume"]
        else:
            value = (orders["Data"][0]["Buy"][0]["Price"])

            if orders["Data"][0]["Buy"][0]["Total"] > volume*value:
                volume = volume*value
            else:
                volume = orders["Data"][0]["Buy"][0]["Total"]
            print("buy" + str(volume) + "min" + str(minimum_transaction[c2]))
            if volume < minimum_transaction[c2]:
                    break
        value = (1 - transaction_fee) * value

        print("BUY ORDER" + str(orders["Data"][0]["Buy"][0]))
        print("SELL ORDER" + str(orders["Data"][0]["Sell"][0]))
        print(value, volume)
    print(item, valid)

print(calls)
print(len(accumulator))
print(end - start)